# Importaciones generales

In [13]:
%cd ~/codeProjects/pythonProjects/Bayesian-Learning-with-Wasserstein-Barycenters
# %cd D:\CodeProjects\Python\Bayesian-Learning-with-Wasserstein-Barycenters\
# Soy una nueva linea

/home/fmunoz/codeProjects/pythonProjects/Bayesian-Learning-with-Wasserstein-Barycenters


In [14]:
import time
from pathlib import Path

import torch
from torch import linalg as LA

from bwb.distributions import *
from bwb.distributions.data_loaders import *
from bwb.transports import *
from bwb.geodesics import *
from bwb import utils
from bwb import sgdw

In [15]:
from bwb import logging
_log = logging.get_logger("notebook")
logging.set_level(level=logging.DEBUG, name="notebook")
logging.set_level(level=logging.INFO, name="bwb.sgdw")

# Cargar datos

In [16]:
CURRENT_PATH = Path(".")
DATA_PATH = CURRENT_PATH / Path("data")
FACE_PATH = DATA_PATH / Path("face.npy")
FACE_PATH

PosixPath('data/face.npy')

In [17]:
arr = np.load(FACE_PATH)
arr.shape

(161666, 784)

In [18]:
shape = (28, 28)
faces = DistributionDrawDataLoader(arr, shape)
faces

DistributionDrawDataLoader((n_models, n_supp)=(161666, 784))

In [19]:
first_face = faces[0]
first_face

In [20]:
max_images = 36
faces_list = []
for k in range(max_images):
    faces_list.append(faces[k])

In [21]:
utils.plot_list_of_draws(faces_list, max_images=max_images)

## Construir posterior

In [22]:
x = first_face.sample((3,))
x

tensor([417, 275, 689], device='cuda:0')

In [23]:
from bwb.config import config
gen = torch.Generator(device=config.device).manual_seed(2147483647)
gen.initial_seed()
# gen.seed()
gen.initial_seed()

2147483647

In [24]:
%%time
pi_n: ExplicitPosteriorPiN[DistributionDraw] = ExplicitPosteriorPiN().fit(data=x, models=faces)
pi_n

CPU times: user 2.47 ms, sys: 939 µs, total: 3.41 ms
Wall time: 2.69 ms


ExplicitPosteriorPiN(n_data=3, n_models=161666, n_support=161666)

# Calcular baricentros

## Definir $\gamma_k$

Aquí se utiliza una función de la forma
\begin{equation*}
    \gamma_k = \frac{a}{(b^{1/c} + k)^c}
\end{equation*}

Con $a > 0$, $b \geq 0$ y $0.5 < c \leq 1$

La idea es que cuando $k=0$, $\gamma_0 = \frac{a}{b}$ es la proporción entre $a$ y $b$, permitiendo ajustar el valor inicial.

In [25]:
class Gamma:
    def __init__(self, a=1, b=0, c=1):
        self.a = a
        self.b = b
        self.c = c
        
    def __call__(self, k):
        return self.a / (self.b ** (1/self.c) + k) ** self.c

In [26]:
window = 3

params = dict(a=3, b=3.1, c=0.51)
gamma = Gamma(**params)

for t in range(window):
    print(f"{t = }; {gamma(t) = :.8f}")
print()

init = 300
for t in range(init, init+window):
    print(f"{t = }; {gamma(t) = :.8f}")
print()

init = 500
for t in range(init, init+window):
    print(f"{t = }; {gamma(t) = :.8f}")
print()

init = 3_000
for t in range(init, init+window):
    print(f"{t = }; {gamma(t) = :.8f}")
print()

init = 10_000
for t in range(init, init+window):
    print(f"{t = }; {gamma(t) = :.8f}")

t = 0; gamma(t) = 0.96774194
t = 1; gamma(t) = 0.91809703
t = 2; gamma(t) = 0.87530574

t = 300; gamma(t) = 0.16110320
t = 301; gamma(t) = 0.16083812
t = 302; gamma(t) = 0.16057432

t = 500; gamma(t) = 0.12491404
t = 501; gamma(t) = 0.12478911
t = 502; gamma(t) = 0.12466455

t = 3000; gamma(t) = 0.05047912
t = 3001; gamma(t) = 0.05047056
t = 3002; gamma(t) = 0.05046201

t = 10000; gamma(t) = 0.02734751
t = 10001; gamma(t) = 0.02734611
t = 10002; gamma(t) = 0.02734472


## Baricentro con 50 caras, 3 datos

In [27]:
faces = DistributionDrawDataLoader(arr[:100, :], shape)
faces

DistributionDrawDataLoader((n_models, n_supp)=(100, 784))

In [28]:
x = first_face.sample((3,))
x

tensor([611, 501, 125], device='cuda:0')

In [29]:
pi_n: ExplicitPosteriorPiN[DistributionDraw] = ExplicitPosteriorPiN().fit(data=x, models=faces)
pi_n

ExplicitPosteriorPiN(n_data=3, n_models=100, n_support=100)

In [30]:
probs = pi_n.probabilities_
torch.round(pi_n.probabilities_[torch.round(probs, decimals=4) > 0] * 100, decimals=2)

tensor([2.5850e+01, 1.5300e+00, 1.0000e-02, 9.1000e-01, 1.0800e+00, 2.1830e+01,
        1.9820e+01, 1.9700e+00, 3.9000e-01, 1.8100e+00, 1.0370e+01, 1.4440e+01],
       device='cuda:0', dtype=torch.float64)

In [31]:
transport = EMDTransport(max_iter=250_000)

In [32]:
X_k, m, pos_hist, samples_hist = sgdw.compute_bwb_discrete_distribution(
    transport=transport,
    posterior=pi_n,
    learning_rate=Gamma(**params),
    batch_size=1,
    alpha=0.1,
    tol=0.,
    max_iter=500,
    max_time=30,
    position_history=True,
    samples_posterior_history=True,
    report_every=20
)

2023-08-18 17:12:20,053: INFO [bwb.sgdw:45]
> dtype = torch.float64, device = device(type='cuda', index=0)
2023-08-18 17:12:20,278: INFO [bwb.sgdw:51]
> total init weights: 4939
2023-08-18 17:12:20,279: INFO [bwb.sgdw:72]
> ============ k = 0, t = 0.0000 [seg], Δt = 0.0000 [ms], Δt per iter. = 0.0002 [ms/iter] ============
2023-08-18 17:12:26,144: INFO [bwb.sgdw:72]
> ============ k = 20, t = 5.8652 [seg], Δt = 292.2466 [ms], Δt per iter. = 279.2972 [ms/iter] ============
2023-08-18 17:12:31,657: INFO [bwb.sgdw:72]
> ============ k = 40, t = 11.3780 [seg], Δt = 267.6003 [ms], Δt per iter. = 277.5111 [ms/iter] ============
2023-08-18 17:12:37,038: INFO [bwb.sgdw:72]
> ============ k = 60, t = 16.7584 [seg], Δt = 264.6067 [ms], Δt per iter. = 274.7273 [ms/iter] ============
2023-08-18 17:12:42,672: INFO [bwb.sgdw:72]
> ============ k = 80, t = 22.3929 [seg], Δt = 311.6219 [ms], Δt per iter. = 276.4555 [ms/iter] ============
2023-08-18 17:12:48,277: INFO [bwb.sgdw:72]
> ============ k = 1

### Primeras iteraciones

In [33]:
utils.plot_list_of_draws([DistributionDraw(X, m, shape) for X in pos_hist[:36]], max_images=36)

In [34]:
utils.plot_list_of_draws([x[0] for x in samples_hist], max_images=33)

### Últimas iteraciones

In [35]:
utils.plot_list_of_draws([DistributionDraw(X, m, shape) for X in pos_hist[-36:]], max_images=36)

In [36]:
utils.plot_list_of_draws([x[0] for x in samples_hist[-33:]], max_images=33)

# Baricentro usando métodos convolucionales, con 50 caras, 3 datos

In [37]:
mu_k, dist_hist, samples_hist = sgdw.compute_bwb_distribution_draw(
    posterior=pi_n,
    learning_rate=Gamma(**params),
    reg=3e-3,
    max_iter=500,
    max_time=30,
    distribution_history=True,
    samples_posterior_history=True,
)

2023-08-18 17:13:36,681: INFO [bwb.sgdw:152]
> dtype = torch.float64, device = device(type='cuda', index=0)
2023-08-18 17:13:36,683: INFO [bwb.sgdw:173]
> ============ k = 0, t = 0.0000 [seg], Δt = 0.0000 [ms], Δt per iter. = 0.0002 [ms/iter] ============
2023-08-18 17:13:37,767: INFO [bwb.sgdw:173]
> ============ k = 10, t = 1.0836 [seg], Δt = 146.8246 [ms], Δt per iter. = 98.5088 [ms/iter] ============
2023-08-18 17:13:39,349: INFO [bwb.sgdw:173]
> ============ k = 20, t = 2.6660 [seg], Δt = 138.8495 [ms], Δt per iter. = 126.9515 [ms/iter] ============
2023-08-18 17:13:40,728: INFO [bwb.sgdw:173]
> ============ k = 30, t = 4.0453 [seg], Δt = 185.0882 [ms], Δt per iter. = 130.4927 [ms/iter] ============
2023-08-18 17:13:42,349: INFO [bwb.sgdw:173]
> ============ k = 40, t = 5.6661 [seg], Δt = 147.0311 [ms], Δt per iter. = 138.1980 [ms/iter] ============
2023-08-18 17:13:43,891: INFO [bwb.sgdw:173]
> ============ k = 50, t = 7.2080 [seg], Δt = 153.0335 [ms], Δt per iter. = 141.3334 [ms

In [38]:
max_images = 38
utils.plot_list_of_draws(dist_hist, max_images=max_images)

In [39]:
utils.plot_list_of_draws([x[0] for x in samples_hist], max_images=max_images)

In [40]:
utils.plot_list_of_draws(dist_hist[-max_images:], max_images=max_images)

In [41]:
utils.plot_list_of_draws([x[0] for x in samples_hist[-max_images:]], max_images=max_images)

# Baricentro usando métodos convolucionales y proyección sobre la variedad, con 50 caras, 3 datos

In [44]:
import torch
from wgan_gp.wgan_gp_vae.model import Generator, Encoder
from wgan_gp.wgan_gp_vae.utils import load_checkpoint
from wgan_gp.wgan_gp_vae.utils import ProjectorOnManifold
import torchvision.transforms as transforms


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LATENT_DIM = 100
CHANNELS_IMG = 1
NUM_FILTERS = [256, 128, 64, 32]

G = Generator(LATENT_DIM, CHANNELS_IMG, NUM_FILTERS).to(device)
E = Encoder(LATENT_DIM, CHANNELS_IMG, NUM_FILTERS[::-1]).to(device)

CURR_PATH = Path(".")
NETS_PATH = CURR_PATH / "wgan_gp" / "networks" 
FACE_PATH = NETS_PATH / "face"

DATA_PATH = CURR_PATH / "data" / "face.npy"

load_checkpoint(G, FACE_PATH, "generator", device)
load_checkpoint(E, FACE_PATH, "encoder", device)


proj = ProjectorOnManifold(
    E, G, 
    transform_in=transforms.Compose(
            [
                # From pdf to grayscale
                transforms.Lambda(lambda x: x / torch.max(x)),
                transforms.ToPILImage(),
                transforms.Resize((64, 64)),
                transforms.ToTensor(),
                transforms.Normalize(
                    [0.5 for _ in range(1)],
                    [0.5 for _ in range(1)],
                ),
            ]
        ),
    transform_out=transforms.Compose(
            [
                # Ensure the range is in [0, 1]
                transforms.Lambda(lambda x: x - torch.min(x)),
                transforms.Lambda(lambda x: x / torch.max(x)),
                transforms.ToPILImage(),
                transforms.Resize((28, 28)),
                transforms.ToTensor(),
                transforms.Lambda(lambda x: x / torch.sum(x)),
                transforms.Lambda(lambda x: x.squeeze(0)),
            ]
        ))

In [45]:
mu_k, dist_hist, samples_hist = sgdw.compute_bwb_distribution_draw_projected(
    posterior=pi_n,
    projector=proj,
    learning_rate=Gamma(**params),
    reg=2e-3,
    max_iter=500,
    max_time=30,
    distribution_history=True,
    samples_posterior_history=True,
)

2023-08-18 17:14:35,675: INFO [bwb.sgdw:240]
> dtype = torch.float64, device = device(type='cuda', index=0)
2023-08-18 17:14:35,676: INFO [bwb.sgdw:261]
> ============ k = 0, t = 0.0000 [seg], Δt = 0.0000 [ms], Δt per iter. = 0.0000 [ms/iter] ============
2023-08-18 17:14:38,708: INFO [bwb.sgdw:261]
> ============ k = 10, t = 3.0322 [seg], Δt = 271.0130 [ms], Δt per iter. = 275.6581 [ms/iter] ============
2023-08-18 17:14:41,127: INFO [bwb.sgdw:261]
> ============ k = 20, t = 5.4510 [seg], Δt = 268.9559 [ms], Δt per iter. = 259.5738 [ms/iter] ============
2023-08-18 17:14:43,336: INFO [bwb.sgdw:261]
> ============ k = 30, t = 7.6605 [seg], Δt = 100.5549 [ms], Δt per iter. = 247.1122 [ms/iter] ============
2023-08-18 17:14:46,277: INFO [bwb.sgdw:261]
> ============ k = 40, t = 10.6017 [seg], Δt = 222.4026 [ms], Δt per iter. = 258.5793 [ms/iter] ============
2023-08-18 17:14:49,551: INFO [bwb.sgdw:261]
> ============ k = 50, t = 13.8750 [seg], Δt = 553.1106 [ms], Δt per iter. = 272.0584 

In [46]:
max_images = 17*3
utils.plot_list_of_draws(dist_hist, max_images=max_images)

In [47]:
utils.plot_list_of_draws([x[0] for x in samples_hist], max_images=max_images)

In [48]:
utils.plot_list_of_draws(dist_hist[-max_images:], max_images=max_images)

In [49]:
utils.plot_list_of_draws([x[0] for x in samples_hist[-max_images:]], max_images=max_images)

In [ ]:
%%time
geod = BarycentricProjGeodesic(EMDTransport(max_iter=250_000))
eps = 0.
max_time_iter = 10
max_iter = float("inf")
alpha = 1

# Camino de baricentros
position_historial = []

# Paso 1: samplear un mu_0
mu_0 = pi_n.draw()

# Calcular las masas y ubicaciones
X_k, m = utils.partition(X=mu_0.enumerate_nz_support_(), mu=mu_0.nz_probs, alpha=alpha)
position_historial.append(X_k)

_log.debug(f"{len(X_k) = }")

k = 0
tic, toc = time.time(), time.time()

while k < max_iter and toc - tic < max_time_iter:
    _log.debug("="*10 + f" {k = }, Δt = {toc - tic:.4f} [seg] " + "="*10)
    # Paso 2: Samplear \tilde\mu_k
    t_mu_k: DistributionDraw = pi_n.draw()
    t_X_k, t_m_k = t_mu_k.enumerate_nz_support_(), t_mu_k.nz_probs

    # Calcular transporte óptimo
    geod.fit(
        Xs=X_k, mu_s=m,
        Xt=t_X_k, mu_t=t_m_k,
    )
    T_X_k = geod.transport.transform(X_k)
    _log.debug(f"{T_X_k.shape = }")

    # Calcular la distribución de mu_{k+1}
    gamma_k = gamma(k); _log.debug(f"{gamma_k = :.6f}")
    X_kp1, _ = geod.interpolate(gamma_k)
    _log.debug(f"{X_kp1.shape = }")

    # Calcular la distancia de Wasserstein
    diff = X_k - T_X_k
    dist = float((gamma_k ** 2) * torch.sum(m * LA.norm(diff, dim=1) ** 2))
    _log.debug(f"{dist = :.8f}")

    while dist < eps:
        break

    position_historial.append(X_kp1)

    k += 1
    X_k = X_kp1
    toc = time.time()

In [ ]:
%%time
eps = 0.
alpha = 1./10
transp = EMDTransport()
max_time_iter = 10 # 60 * 60  # Una hora
max_iter = float("inf")

# Camino de baricentros
position_historial = []

# Paso 1: samplear un mu_0
mu_0 = pi_n.draw()

# Calcular las masas y ubicaciones
X, m_ = [], []
min_w, max_w = mu_0.nz_probs.min(), mu_0.nz_probs.max()

for x, w, n in zip(mu_0.enumerate_nz_support_(), mu_0.nz_probs,
                   torch.ceil(alpha * mu_0.nz_probs / min_w).to(int)):
    for _ in range(n):
        X.append(x.reshape(1, -1))
        m_.append(w / n)

X_k = torch.cat(X, 0)
m = torch.as_tensor(m_, dtype=mu_0.dtype, device=mu_0.device)
position_historial.append(X_k)

_log.debug(f"{len(X_k) = }")

k = 0

tic, toc = time.time(), time.time()

while k < max_iter and toc - tic < max_time_iter:
    _log.debug("\n" + "="*10 + f" {k = }, Δt = {toc - tic:.4f} [seg] " + "="*10)
    # Paso 2: Samplear \tilde\mu_k
    t_mu_k: DistributionDraw = pi_n.draw()
    t_X_k, t_m_k = t_mu_k.enumerate_nz_support_(), t_mu_k.nz_probs

    # Calcular transporte óptimo
    transp.fit(
        Xs=X_k, mu_s=m,
        Xt=t_X_k, mu_t=t_m_k,
    )
    T_X_k = transp.transform(X_k)
    _log.debug(f"{T_X_k.shape = }")

    # Calcular la distribución de mu_{k+1}
    gamma_k = gamma(k); _log.debug(f"{gamma_k = :.6f}")
    X_kp1 = (1 - gamma_k) * X_k + gamma_k * T_X_k  # Basta con calcular las transformaciones, porque los pesos son los mismos
    _log.debug(f"{X_kp1.shape = }")

    # Calcular la distancia de Wasserstein
    diff = X_k - T_X_k
    dist = float((gamma_k ** 2) * torch.sum(m * LA.norm(diff, dim=1) ** 2))
    _log.debug(f"{dist = :.8f}")

    while dist < eps:
        break

    position_historial.append(X_kp1)

    k += 1
    X_k = X_kp1
    toc = time.time()

In [ ]:
%%time
reshaped_positions = []
n_coord, n_dim = position_historial[0].shape
for p in position_historial:
    p_reshape = p.reshape(n_coord, 1, n_dim)
    reshaped_positions.append(p_reshape)

In [ ]:
%%time
positions_historial_batch = torch.cat(reshaped_positions, dim=1)

In [ ]:
positions_historial_batch.shape

In [ ]:
import pickle
from pathlib import Path
import os


SAVE = False


PICKLES_PATH = Path("./pickles")

POSITION_HISTORIAL_PATH = PICKLES_PATH / "postion_historial.dat"
WEIGHTS_PATH = PICKLES_PATH / "weights.dat"

if not PICKLES_PATH.exists(): PICKLES_PATH.mkdir()

if SAVE:
    if POSITION_HISTORIAL_PATH.exists():
        os.remove(POSITION_HISTORIAL_PATH)

    if WEIGHTS_PATH.exists():
        os.remove(WEIGHTS_PATH)

    with POSITION_HISTORIAL_PATH.open("wb") as f:
        pickle.dump(positions_historial_batch, f)

    with WEIGHTS_PATH.open("wb") as f:
        pickle.dump(m, f)

PICKLES_PATH.exists()

In [ ]:
import pickle
from pathlib import Path

d = 2
PICKLES_PATH = Path("./pickles")
POSITION_HISTORIAL_PATH = PICKLES_PATH / f"postion_historial{d}.dat"
WEIGHTS_PATH = PICKLES_PATH / f"weights{d}.dat"

with POSITION_HISTORIAL_PATH.open("rb") as f:
    new_position_historial = pickle.load(f)

with WEIGHTS_PATH.open("rb") as f:
    m = pickle.load(f)

new_position_historial, m

In [ ]:
len(new_position_historial)

In [ ]:
new_position_historial[0].shape, new_position_historial[1].shape, new_position_historial[2].shape

In [ ]:
new_position_historial = new_position_historial[1:]

In [ ]:
%%time
reshaped_positions = []
n_coord, n_dim = new_position_historial[0].shape
for p in new_position_historial:
    p_reshape = p.reshape(n_coord, 1, n_dim)
    reshaped_positions.append(p_reshape)

In [ ]:
%%time
positions_historial_batch = torch.cat(reshaped_positions, dim=1)

In [ ]:
positions_historial_batch.shape

In [ ]:
m.shape

In [ ]:
# position_historial = new_position_historial
# position_historial[:, 0, :].shape

In [ ]:
# pos_hist_rounded = torch.round(positions_historial_batch)
# pos_hist_ind = (pos_hist_rounded[:, :, 0] * m + pos_hist_rounded[:, :, 1]).to(int)
# pos_hist_ind

In [ ]:
# pos_hist_ind.shape

In [ ]:
# m.shape

In [ ]:
# to_return = torch.zeros((13, 28*28))
#
# for w, p in zip(m, pos_hist_ind):
#     for i, pp in enumerate(p):
#         to_return[i, pp] += w

In [ ]:
# to_return.sum(1)

In [ ]:
%%time
def position_to_weights(positions, weights, shape):

    positions, weights = torch.as_tensor(positions), torch.as_tensor(weights)

    n, m  = shape
    n_coord, n_pos, n_dim = positions.shape

    to_return = torch.zeros((n_pos, n * m))
    pos_rounded = torch.round(positions)
    pos_ind = (pos_rounded[:, :, 0]*m + pos_rounded[:, :, 1]).to(int)

    for w, p in zip(weights, pos_ind):
        for i, pp in enumerate(p):
            to_return[i, pp] += w

    return to_return

# ptw = position_to_weights([[0, 0], [0, 0], [1, 0], [1, 1]], [0.2, 0.3, 0.1, 0.4], (2, 2))
# ptw

ptw = position_to_weights(positions_historial_batch[:, :10, :], m, (28, 28))

In [ ]:
def position_to_list_of_dd(positions, weights, shape, i, j):
    return [DistributionDraw(w, shape) for w in position_to_weights(positions[:, i:j, :], weights, shape)]
list_dd = position_to_list_of_dd(positions_historial_batch[:, :10, :], m, (28, 28), 0, 10)
list_dd

In [ ]:
from bwb.utils import plot_list_of_draws
list_dd = position_to_list_of_dd(positions_historial_batch, m, (28, 28), 0, 36)
plot_list_of_draws(list_dd)

In [ ]:
list_dd = position_to_list_of_dd(positions_historial_batch, m, (28, 28), 36, 72)
plot_list_of_draws(list_dd)

In [ ]:
list_dd = position_to_list_of_dd(positions_historial_batch, m, (28, 28), -37, -1)
plot_list_of_draws(list_dd)

In [ ]:
n_coord, n_pos, n_dim = positions_historial_batch.shape
to_return = torch.zeros((n_pos, 28*28))
pos_rounded = torch.round(positions_historial_batch)
pos_ind = (pos_rounded[:, :, 0]*28 + pos_rounded[:, :, 1]).to(int)

gen = zip(m, pos_ind)
w, p = next(gen)
w.shape, p.shape

In [ ]:
to_return[:, 0].shape

In [ ]:
ptw

In [ ]:
%%time
historial_bsgd = torch.concat([position_to_weights(p, m, (28, 28)).reshape((1, -1)) for p in new_position_historial], 0)

In [ ]:
dddl = DistributionDrawDataLoader(

)

In [ ]:
arr = torch.arange(28*28)
# arr = torch.reshape(arr, (28, 28))

In [ ]:
pos = torch.round(new_position_historial[-1][:10]).to(int)
pos_ind = pos[:, 0] * 28 + pos[:, 1]
# for p in map(tuple, pos):
#     print(arr[p])
# pos_ind

arr[pos_ind]